# Imports

In [13]:
import pandas as pd
import kagglehub
from pathlib import Path
from PIL import Image
import os

# Load dataset

In [14]:
# Download latest version
if os.path.exists(r"C:\Users\desus\.cache\kagglehub\datasets\abhikjha\utk-face-cropped\versions\1"):
    raw_data_path = r"C:\Users\desus\.cache\kagglehub\datasets\abhikjha\utk-face-cropped\versions\1"
    print("Path to dataset files already exists: ", raw_data_path)

else:
    raw_data_path = kagglehub.dataset_download("abhikjha/utk-face-cropped")
    print("Path to dataset files:", raw_data_path)

Path to dataset files already exists:  C:\Users\desus\.cache\kagglehub\datasets\abhikjha\utk-face-cropped\versions\1


> *Filename structure*:
> * First interger -- age
> * Second interger -- male(0)/female(1)
> * Third integer -- ethnicity: white(0)/black(1)/asian(2)/indian(3)/other(4) 

# Create a subset of relevant data

*Ages 40-60, White females*

In [3]:
# Function to search for files with a specific keyword in their names
def search_files(directory, keyword, list):
    for file in Path(directory).rglob(f"*{keyword}*"):
        list.append(file)

In [4]:
# Create subset of only white females between 40 and 59 years old
directory = os.path.join(raw_data_path, r"utkcropped\utkcropped")
print(directory)
filename_list = []
for i in range(40, 61):
    search_files(directory, f"{i}_1_0", filename_list)

C:\Users\desus\.cache\kagglehub\datasets\abhikjha\utk-face-cropped\versions\1\utkcropped\utkcropped


In [5]:
# Create a dataframe of filenames
df = pd.DataFrame(filename_list, columns=['Filename'])
len(df)

904

In [6]:
# Shuffle and keep only 30% of the data
shuffled_df = df.sample(frac=0.5).reset_index(drop=True)

In [7]:
len(shuffled_df)

452

In [8]:
# Save the subset to a new folder
current_dir = os.getcwd()
destination_dir = os.path.join(current_dir, 'data', 'raw_data')
print(destination_dir)

for i in shuffled_df.index:
    source=shuffled_df.iloc[i, 0]
    img = Image.open(source)
    img.save(os.path.join(destination_dir, os.path.basename(source)))


c:\Users\desus\OneDrive - Danmarks Tekniske Universitet\Fall_2025\Cognitive_Modelling\Kareninator\data\raw_data


# Manual check of data

*Remove images that do not fulfill the following criteria* -- 
* Full frontal face
* "White" woman
* Visible eyes and features (no sunglasses)
* High resolution
* No watermarks

Number of images *before* manual removal - 452

Number of images *after* manual removal - 272


# Grayscale images

In [15]:
# Save the subset to a new folder
current_dir = os.getcwd()
destination_dir = os.path.join(current_dir, 'data', 'raw_data')

grayscale_img_dir = os.path.join(current_dir, 'data', 'preprocessed_data')

for img in os.listdir(destination_dir):
    gray_img = Image.open(os.path.join(current_dir, 'data', 'raw_data', img)).convert('L')
    gray_img_resized = gray_img.resize((100, 100), Image.LANCZOS)
    gray_img_resized.save(os.path.join(grayscale_img_dir, img))